In [ ]:
#r "./../../../../../../public/src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
using System;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Solution;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.BoSSSpad;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Control;
using BoSSS.Solution.XNSECommon;
using BoSSS.Solution.NSECommon;
using BoSSS.Application.XNSE_Solver.LoadBalancing;
using BoSSS.Solution.LevelSetTools;
using BoSSS.Solution.XdgTimestepping;

using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [ ]:
// set parameterz
int[] core_sweep = {4};
int[] PolyDegS = new int[] {3};
int[] ResArray = new int[] {10};
int MemoryPerCore = 2000;

In [ ]:
bool useAMR = true;
bool useLoadBal = true;
int NoOfTimeSteps = 1;
bool Steady = true;
bool IncludeConvection = true;
var Gshape = Shape.Cube;

In [ ]:
ExecutionQueues[1]

DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,BatchInstructionDir,AllowedDatabasesPaths
C:\Users\weber\AppData\Local\BoSSS-LocalJobs,False,LocalPC,dotnet,<null>,"[ { D:\trash_db == : LocalMountPath: D:\trash_db, PathAtRemote: } ]"


In [ ]:
// ==================================
// setup Client & Workflow & Database
// ==================================
/*
var myBatch = (SlurmClient)ExecutionQueues[3];
var AddSbatchCmds = new List<string>();
AddSbatchCmds.AddRange(new string[]{"#SBATCH -p test24", "#SBATCH -C avx512", "#SBATCH --mem-per-cpu="+MemoryPerCore});
myBatch.AdditionalBatchCommands = AddSbatchCmds.ToArray();
myBatch.AdditionalBatchCommands
*/

In [ ]:
static var myBatch = (MiniBatchProcessorClient)ExecutionQueues[1];
MiniBatchProcessor.Server.StartIfNotRunning();

In [ ]:
string WFlowName = "rotCube+Inlet_test";
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.Init(WFlowName);
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();

In [ ]:
/*
myBatch.AllowedDatabasesPaths.Clear();
//string dirname = "DB_rotCubePaper";
//string dirname = "DB_rotSphere_CoreScaling";
//string dirname = "DB_rotSphere_DOFScaling";
string dirname ="DB_rotCubePaper";
string winpath = @"W:\work\scratch\jw52xeqa\"+dirname;
string remotepath = @"/work/scratch/jw52xeqa/"+dirname;
//string winpath = @"\\hpccluster\hpccluster-scratch\"+dirname;
var myDB = OpenOrCreateDatabase(winpath);
//myBatch.AllowedDatabasesPaths.Add(new AllowedDatabasesPair(winpath,remotepath)); myBatch.AllowedDatabasesPaths
*/

In [ ]:
static var pair = myBatch.AllowedDatabasesPaths.Pick(0);
static string DBName = @"\"+"DB_rotCubePaper";
static string localpath=pair.LocalMountPath+DBName;
string remotepath=pair.PathAtRemote+DBName;
static var myDB = OpenOrCreateDatabase(localpath);

Opening existing database 'D:\trash_db\DB_rotCubePaper'.


In [ ]:
static class Utils {
    // DOF per cell in 3D
    public static int Np(int p) {
        return (p*p*p + 6*p*p + 11*p + 6)/6;
    }
}

using 

In [ ]:
double xMax = 4.0, yMax = 1.0, zMax = 1.0;
double xMin = -2.0, yMin = -1.0,zMin = -1.0;

double xMax = 1.0, yMax = 1.0, zMax=1.0;
double xMin = -1.0, yMin = -1.0,zMin = -1.0;

In [ ]:
List<IGridInfo> Grids = new List<IGridInfo>();
foreach(var Res in ResArray){
    int Stretching = (int)Math.Floor(Math.Abs(xMax-xMin)/Math.Abs(yMax-yMin));
    var _xNodes = GenericBlas.Linspace(xMin, xMax, Stretching*Res + 1);
    var _yNodes = GenericBlas.Linspace(yMin, yMax, Res + 1);
    var _zNodes = GenericBlas.Linspace(zMin, zMax, Res + 1);

    GridCommons grd;
    string gname = "RotBenchmarkGrid";
    
    var tmp = new List<IGridInfo>();
    foreach(var grid in myDB.Grids){
        try{
            bool IsMatch = grid.Name.Equals(gname)&&grid.NumberOfCells==_xNodes.Length*_yNodes.Length*_zNodes.Length;
            if(IsMatch) tmp.Add(grid);
        }
        catch(Exception ex) {
            Console.WriteLine(ex.Message);
        }
    }
    if(tmp.Count()>=1){
        Console.WriteLine("Grid found: "+tmp.Pick(0).Name);
        Grids.Add(tmp.Pick(0));
        continue;
    }
    
    grd = Grid3D.Cartesian3DGrid(_xNodes, _yNodes, _zNodes);
    grd.Name = gname;
    //grd.AddPredefinedPartitioning("debug", MakeDebugPart);

    grd.EdgeTagNames.Add(1, "Velocity_inlet");
    grd.EdgeTagNames.Add(2, "Wall");
    grd.EdgeTagNames.Add(3, "Pressure_Outlet");

    grd.DefineEdgeTags(delegate (double[] _X) {
        var X = _X;
        double x, y, z;
        x = X[0];
        y = X[1];
        z = X[2];
        if(Math.Abs(x-xMin)<1E-8)
            return 1;
        else
            return 3;
    });
    myDB.SaveGrid(ref grd);
    Grids.Add(grd);
} Grids

Grid Edge Tags changed.
An equivalent grid (98636c04-a045-4131-8c07-8d9a58997272) is already present in the database -- the grid will not be saved.


#0: { Guid = 98636c04-a045-4131-8c07-8d9a58997272; Name = RotBenchmarkGrid; Cell Count = 3000; Dim = 3 }


In [ ]:
int SpaceDim = 3;
double _partRad = 0.3800;

In [ ]:
_partRad/(2/(double)ResArray[0]/4)

7.6

In [ ]:
Action<XNSE_Control, int> SetOptionsResFields = delegate (XNSE_Control ctrl, int k){
    ctrl.FieldOptions.Add("Residual-MomentumX", new FieldOpts() {
        Degree = k,
        SaveToDB = FieldOpts.SaveToDBOpt.TRUE
    });
    ctrl.FieldOptions.Add("Residual-MomentumY", new FieldOpts() {
        Degree = k,
        SaveToDB = FieldOpts.SaveToDBOpt.TRUE
    });
    ctrl.FieldOptions.Add("Residual-MomentumZ", new FieldOpts() {
        Degree = k,
        SaveToDB = FieldOpts.SaveToDBOpt.TRUE
    });
    ctrl.FieldOptions.Add("Residual-ContiEq", new FieldOpts() {
        Degree = k-1,
        SaveToDB = FieldOpts.SaveToDBOpt.TRUE
    });
};

In [ ]:
Func<IGridInfo, int, double, LinearSolverCode, XNSE_Control> GenXNSECtrl = delegate(IGridInfo grd, int k, double Viscosity, LinearSolverCode solver ){
    XNSE_Control C = new XNSE_Control();
    // basic database options
    // ======================
    C.AlternateDbPaths = new[] {
        (localpath, ""),
        (remotepath,"")};
    //C.DbPath=@"\\hpccluster\hpccluster-scratch\DB_rotSphereBenchmark";
    C.savetodb = true;
    int J  = grd.NumberOfCells;
    C.SessionName = string.Format("J{0}_k{1}_t{2}_{3}", J, k,NoOfTimeSteps,solver.ToString());
    if(IncludeConvection){
        C.SessionName += "_NSE";
        C.Tags.Add("NSE");
    } else {
        C.SessionName += "_Stokes";
        C.Tags.Add("Stokes");
    }
    C.Tags.Add(SpaceDim + "D");
    if(Steady)C.Tags.Add("steady");
    else C.Tags.Add("transient");

    // DG degrees
    // ==========
    C.SetFieldOptions(k, Math.Max(k, 2));
    SetOptionsResFields(C,k);
    C.saveperiod = 1;
    //C.TracingNamespaces = "*";

    C.GridGuid = grd.ID;
    C.GridPartType = GridPartType.clusterHilbert;
    C.DynamicLoadbalancing_ClassifierType = ClassifierType.CutCells;
    C.DynamicLoadBalancing_On = useLoadBal;
    C.DynamicLoadBalancing_RedistributeAtStartup = true;
    C.DynamicLoadBalancing_Period = 1;
    C.DynamicLoadBalancing_ImbalanceThreshold = 0.1;


    // Physical Parameters
    // ===================
    const double rhoA = 1;
    const double Re = 100;
    double muA = Viscosity;
    
    double partRad = _partRad;
    double d_hyd = 2*partRad;
    double anglev = Re*muA/rhoA/d_hyd;
    double VelocityIn = Re*muA/rhoA/d_hyd;
    double[] pos = new double[SpaceDim];
    double ts = Math.PI/anglev/NoOfTimeSteps;
    double inletdelay = 5*ts;

    C.PhysicalParameters.IncludeConvection = IncludeConvection;
    C.PhysicalParameters.Material = true;
    C.PhysicalParameters.rho_A = rhoA;
    C.PhysicalParameters.mu_A = muA;

    C.Rigidbody.SetParameters(pos,0.0,partRad,SpaceDim);
    //C.Rigidbody.SetParameters(pos,anglev,partRad,SpaceDim);
    C.Rigidbody.SpecifyShape(Gshape);
    C.Rigidbody.SetRotationAxis("x");
    C.AddInitialValue(VariableNames.LevelSetCGidx(0), new Formula("X => -1"));
    //C.UseImmersedBoundary = true;
    
    C.AddInitialValue("Pressure", new Formula(@"X => 0"));
    C.AddBoundaryValue("Pressure_Outlet");
    //var DelayedInlet = new Formula($"(X,t) => {VelocityIn}*(double)(t<={inletdelay}?(Math.Sin((t-{inletdelay}/2)/{inletdelay}*{Math.PI})+1)/2:1)", true);
    var DelayedInlet = new Formula($"(X,t) => {VelocityIn}", true);
    C.AddBoundaryValue("Velocity_inlet", "VelocityX", DelayedInlet);
    C.AddInitialValue("VelocityX", DelayedInlet);

    C.CutCellQuadratureType = BoSSS.Foundation.XDG.XQuadFactoryHelper.MomentFittingVariants.Saye;
    C.UseSchurBlockPrec = true;
    C.AgglomerationThreshold = 0.1;
    C.AdvancedDiscretizationOptions.ViscosityMode = ViscosityMode.FullySymmetric;
    C.Option_LevelSetEvolution2 = LevelSetEvolution.Prescribed;
    C.Option_LevelSetEvolution = LevelSetEvolution.None;
    C.Timestepper_LevelSetHandling = LevelSetHandling.None;
    C.LinearSolver.NoOfMultigridLevels = 4;
    C.LinearSolver.ConvergenceCriterion = 1E-8;
    C.LinearSolver.MaxSolverIterations = 200;
    C.LinearSolver.MaxKrylovDim = 50;
    C.LinearSolver.TargetBlockSize = 10000;
    C.LinearSolver.verbose = true;
    C.LinearSolver.SolverCode = solver;
    C.NonLinearSolver.SolverCode = NonLinearSolverCode.Newton;
    //C.NonLinearSolver.ConvergenceCriterion = 1E-5;
    C.NonLinearSolver.MaxSolverIterations = 6;
    C.NonLinearSolver.verbose = true;

    C.AdaptiveMeshRefinement = useAMR;
    if (useAMR) {
        C.SetMaximalRefinementLevel(2);
        C.AMR_startUpSweeps = 0;
    }

    // Timestepping
    // ============
    double dt = -1;
    if(Steady){
        C.TimesteppingMode = AppControl._TimesteppingMode.Steady;
        dt = 1000;
        C.NoOfTimesteps = 1;
    } else {
        C.TimesteppingMode = AppControl._TimesteppingMode.Transient;        
        dt = ts;        
        C.NoOfTimesteps = NoOfTimeSteps;
    }
    C.TimeSteppingScheme = TimeSteppingScheme.BDF2;
    C.dtFixed = dt;
    return C;
};

In [ ]:
var ViscositySweep = new double[]{1E-3};
List<XNSE_Control> controls = new List<XNSE_Control>();
var solvers = new LinearSolverCode[]{LinearSolverCode.classic_pardiso, LinearSolverCode.exp_Kcycle_schwarz};
foreach(var solver in solvers){
    foreach(IGridInfo grd in Grids){
        foreach(int k in PolyDegS){
            foreach(double v in ViscositySweep)
                controls.Add(GenXNSECtrl(grd,k,v,solver));
        }
    }
} controls.Select(s=>s.SessionName)

index,value
0,J3000_k3_t1_classic_pardiso_NSE
1,J3000_k3_t1_exp_Kcycle_schwarz_NSE


In [ ]:
controls.Select(s=>s.SessionName)

index,value
0,J3000_k3_t1_classic_pardiso_NSE
1,J3000_k3_t1_exp_Kcycle_schwarz_NSE


In [ ]:
static Func<XNSE_Control,int,int> NodeRegression =  delegate (XNSE_Control thisctrl, int cores) {
    int Ncells = myDB.Grids.Where(g=>g.ID.Equals(thisctrl.GridGuid)).Pick(0).NumberOfCells;
    thisctrl.FieldOptions.TryGetValue("Velocity*",out FieldOpts Foptions);
    int k = Foptions.Degree;
    int DOFperCell = Utils.Np(k)*3+Utils.Np(k-1);
    int DOFperCore = DOFperCell*Ncells/cores;
    double mempercore = -9E-7*DOFperCore*DOFperCore+3.84E-1*DOFperCore+13500;
    double memoryneed = mempercore * cores;
    double memorypernode = 384*1024; // MB
    int nodes2allocate = (int)Math.Ceiling(memoryneed / memorypernode);
    
    // these 2 lines make it a power of 2
    double bla = Math.Ceiling(Math.Log(nodes2allocate)/Math.Log(2));
    nodes2allocate = (int)Math.Pow(2,bla);

    return nodes2allocate;
};

In [ ]:
NodeRegression(controls.Pick(0),64)

4

In [ ]:
foreach(var ctrl in controls){
    string sessname = ctrl.SessionName;
    foreach(int cores in core_sweep){
        ctrl.SessionName = sessname + "_c"+cores+"_mue_"+ctrl.PhysicalParameters.mu_A;
        var aJob   = new Job("rotting_"+Gshape+ctrl.SessionName,typeof(XNSE));
        aJob.SetControlObject(ctrl);
        aJob.NumberOfMPIProcs         = cores;
        aJob.ExecutionTime            = "3:00:00";
        aJob.UseComputeNodesExclusive = true;

        aJob.Activate(myBatch);
    }
}

Note: found 1 successful deployment(s), but job is configured to require a successful result session ('this.SessionReqForSuccess' is true), and none is found. 0 sessions correlated to this job fount in total.
Deploying job rotting_CubeJ3000_k3_t1_classic_pardiso_NSE_c4_mue_0.001 ... 
Deploying executables and additional files ...
Deployment directory: C:\Users\weber\AppData\Local\BoSSS-LocalJobs\rotCube+Inlet_test-XNSE_Solver2021Nov04_124402
copied 49 files.
   written file: control.obj
deployment finished.
Mini batch processor is already running.

Note: found 1 successful deployment(s), but job is configured to require a successful result session ('this.SessionReqForSuccess' is true), and none is found. 0 sessions correlated to this job fount in total.
Deploying job rotting_CubeJ3000_k3_t1_exp_Kcycle_schwarz_NSE_c4_mue_0.001 ... 
Deploying executables and additional files ...
Deployment directory: C:\Users\weber\AppData\Local\BoSSS-LocalJobs\rotCube+Inlet_test-XNSE_Solver2021Nov04_124